In [1]:
import sys
sys.path.append('/home/ak/Documents/Research/hsmm/') 


In [2]:
import pandas as pd 
import numpy as np 
np.set_printoptions(precision=3, suppress=True)
from scipy import stats
import matplotlib.pyplot as plt
from scipy import signal
from sklearn import preprocessing as prep
import time

from hsmm_core.hmm import * 
from hsmm_core.prediction_engines import * 
from hsmm_core.consts import *
from hsmm_core.statistics import *
from hsmm_core.observation_models import *

In [3]:
# font options
font = {
    #'family' : 'normal',
    #'weight' : 'bold',
    'size'   : 16
}

plt.rc('font', **font)
plt.rc('lines', linewidth=2)
#matplotlib.rc('text', usetex=True)
#matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

In [4]:
##format all floats to be to two points only##
float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})

In [5]:
n_hidden_states = 3
startprob = np.array([0.5, 0.2, 0.3])

transmat = np.array([[0.2, 0.4, 0.4], [0.5, 0.25, 0.25], [0.5, 0.4, 0.1]])



init_params = {
    "obs_model_params": {
                                'obs_model_name': 'ExpUniGauss',
                                'em_init_method': InitialisationMethod.cluster

    },
    "hidden_model_params": {
                                'no_hidden_states': n_hidden_states,
                                'pi':startprob,
                                'tpm': transmat,
                                'em_init_method': InitialisationMethod.uniform
    },
    "update_tag": 'tpsml'
}
obs_model_init = {
    'sigmas': np.array([0.5, 0.003, 0.5]),
    'lambdas': np.array([1., 0.01, 1.]),
    'weights': np.array([0.2, 0.5, 0.3]),
}
obs_model = ExpUniGauss(n_hidden_states)

# obs_model.set_up_initials(priors=obs_model_init)

the_hmm = hmm_engine(obs_model, n_hidden_states)


priors = {'tpm': transmat, 'pi': startprob}
priors.update(obs_model_init)

the_hmm.set_up_initials(priors=priors)

data=performance_statistics(hmm=the_hmm).generate_observations(sequence_length=100, no_paths=10)


In [6]:
data.shape

(10, 100, 2)

In [8]:
pd.DataFrame(data[1,:,])

,0,1
0,0.360824,0.000000
1,-0.875853,0.622444
2,0.003098,6.832262
3,26.013624,0.000000
4,-0.869902,2.700484
5,0.166253,0.157172
6,0.725073,0.000000
7,0.001587,4.070081
8,-0.119168,0.124496
9,18.544381,0.000000


In [7]:
n_hidden_states = 2
no_observations = 1000

obs_model_init = {
    'sigmas': np.array([0.6, 0.013]),
    'lambdas': np.array([1., 0.01]),
    'weights': np.array([0.5, 0.5]),
}

    
tpm_2s = np.array([[0.6, 0.4], [0.3, 0.7]])
startprob_2s = np.array([0.7, 0.3])
obs_model_2s = ExpUniGauss(n_hidden_states)

# obs_model.set_up_initials(priors=obs_model_init)

the_hmm_two_states = hmm_engine(obs_model_2s, n_hidden_states)

priors = {'tpm': tpm_2s, 'pi': startprob_2s}
priors.update(obs_model_init)

the_hmm.set_up_initials(priors=priors)

data_2states=performance_statistics(hmm=the_hmm).generate_observations(sequence_length=no_observations, no_paths=100)

In [10]:

import datetime as dt
open_time=  dt.time(8,00,00)
close_time=  dt.time(16,30,00)


In [11]:
data_2states.shape
#pd.DataFrame(data[1,:,:])

(100, 1000, 2)

In [17]:
test=pd.DataFrame(data_2states[1,:,:],columns=['ReturnTradedPrice', 'Duration'])


In [20]:
import datetime
def timestamps_(sequence_length):
    dateTimeA = datetime.datetime.combine(datetime.date.today(), dt.time(8, 00, 00))
    dateTimeB = datetime.datetime.combine(datetime.date.today(), dt.time(16, 30, 00))
    delta_ = dt.timedelta(days=0,seconds=np.abs((dateTimeA - dateTimeB).total_seconds())/ sequence_length,microseconds=1.06)
    timestamps_= [(dateTimeA+ delta_*x).strftime('%H:%M:%S:%f') for x in range (0,sequence_length)]
    return timestamps_

In [21]:
test['TradedTime']=timestamps_(1000)

In [22]:
test

,ReturnTradedPrice,Duration,TradedTime
0,87.287145,0.000000,08:00:00:000000
1,0.003282,15.342952,08:00:30:600001
2,0.227106,1.632876,08:01:01:200002
3,1.883758,0.000000,08:01:31:800003
4,0.816687,0.000000,08:02:02:400004
5,0.542905,0.000000,08:02:33:000005
6,-0.003044,82.341536,08:03:03:600006
7,27.492284,0.000000,08:03:34:200007
8,0.376294,1.853466,08:04:04:800008
9,0.003806,46.933710,08:04:35:400009


In [ ]:
for key in range(0,1):
    df_features_all=pd.concat([features[fkeys[key]][0], features[fkeys[key]][2], 
                                    features[fkeys[key]][3]],axis=1).dropna()
    df_features_all.drop('fischer_score_dweight', axis=1, inplace=True)
    train_labels =data_dic[fkeys[key]].dropna()['trend_label_quantiles']
    diff= len(df_features_all) -len(train_labels)
    
    #########
    X= df_features_all.iloc[diff:]
    n_features = X.shape[1]
    
    y= train_labels
    n_classes = train_labels.shape[0]
    y_minus = (y==-1).astype(int)
    y_zeros = (y==0).astype(int)
    y_plus = (y==1).astype(int)
    ####splitting data into train and test sets####
    #### normalise
    X_train, X_test, y_train_minus, y_test_minus = \
    train_test_split(sc.fit_transform(X), y_minus, test_size=0.25, random_state=0) #splitting training and test labels for y_minus
    X_train, X_test, y_train_plus, y_test_plus = \
    train_test_split(sc.fit_transform(X), y_plus, test_size=0.25, random_state=0) #splitting training and test labels for y_minus
    t0 = time.time()
    print t0
   
#

In [ ]:
##pick number of combinations of features##
no_i=2
df2= [df_features_all[list(pair)]for pair in list(iter.combinations(df_features_all, no_i)) ]
##list to store results###
results_list = []

In [ ]:
for key in range(0,3):
    df_features_all=pd.concat([features[fkeys[key]][0], features[fkeys[key]][2], 
                                    features[fkeys[key]][3]],axis=1).dropna()
    df_features_all.drop('fischer_score_dweight', axis=1, inplace=True)
    train_labels =data_dic[fkeys[key]].dropna()['trend_label_quantiles']
    
    
    for j in range(0,90):
        no_i=2
        df2[j]= [df_features_all[list(pair)]for pair in list(iter.combinations(df_features_all, no_i)) ] 
        #print key+1, 'KEY'
        #print j, 'index of combo of pair of features'
        #print df2[key][j].columns.values
        diff= len(df2[key][j]) -len(train_labels)
        #print'######'
  
        #########
        X= df2[key][j].iloc[diff:]
        y= label_binarize(train_labels, classes=[0,1,2])
        y= train_labels
        n_classes = train_labels.shape[0]
        y_minus = (y==-1).astype(int)
        y_zeros = (y==0).astype(int)
        y_plus = (y==1).astype(int)
        ####splitting data into train and test sets####
        #### normalise
        
        X_train, X_test, y_train_minus, y_test_minus = \
        train_test_split(sc.fit_transform(X), y_minus, test_size=0.25, random_state=0) #splitting training and test labels for y_minus
        
        X_train, X_test, y_train_plus, y_test_plus = \
        train_test_split(sc.fit_transform(X), y_plus, test_size=0.25, random_state=0) #splitting training and test labels for y_minus
        
#         n_classes =y.shape[1]
#         n_samples, n_features = X.shape
#         #### normalise
#         X_train, X_test, y_train, y_test = \
#         train_test_split(sc.fit_transform(X), y, test_size=0.1, random_state=0) #splitting training and test labels for y_minus
# #         parameters = {'kernel':('linear', 'rbf'),'gamma' : np.logspace(-2, 1, 5), 'C':[1, 10]}
        clf = svm.SVC(kernel ='rbf', C=10, gamma='auto', random_state=0, class_weight="balanced", cache_size=1024)
        clf.fit(X_train, y_train_plus)
        #print 'Best parameters %s' %clf.best_params_
        #print 'Cross validation accuracy: mean = %0.3f' % clf.best_score_
        t0 = time.time()
        scores = cross_val_score(clf, X_train, y_train_plus, cv=5,scoring='accuracy', n_jobs=-1)
        #print key +1
        #print "Training Set"
        #print "SVC with rbf kernel -> cross validation accuracy: mean = %0.3f std = %0.3f" % (np.mean(scores), np.std(scores))
        #print "Test Set"
        
        y_pred = clf.predict(X_test)
        #print clf.score(X_test, y_test_plus)  
        test_mse =mean_squared_error(y_test_plus, y_pred)
        test_fbeta= fbeta_score(y_test_plus, y_pred, average='macro', beta=0.5)
        accuracy_ = accuracy_score(y_test_plus, y_pred)
        cnf_matrix=confusion_matrix(y_test_plus, y_pred).astype('float')
        cnf_matrix =cnf_matrix/cnf_matrix.sum(axis=1) #normalised confusion matrix
        #print "R2_score: %3f" %r2_score(y_test_plus, y_pred)
        results_list.append({'Features':df2[key][j].columns.values,'accuracy':accuracy_,'confusion matrix':cnf_matrix,'test_mse':test_mse, 'fbeta':test_fbeta})
        ##### parameters###

    print 'Time passed:'
    print time.time()- t0
    print '####'

In [ ]:
for j in range(0,90):
        no_i=2
        df2[j]= [df_features_all[list(pair)]for pair in list(iter.combinations(df_features_all, no_i)) ] 
        #print key+1, 'KEY'
        #print j, 'index of combo of pair of features'
        #print df2[key][j].columns.values
        diff= len(df2[key][j]) -len(train_labels)
        #print'######'
  
        #########
        X= df2[key][j].iloc[diff:]
        y= label_binarize(train_labels, classes=[0,1,2])
        y= train_labels
        n_classes = train_labels.shape[0]
        y_minus = (y==-1).astype(int)
        y_zeros = (y==0).astype(int)
        y_plus = (y==1).astype(int)
        ####splitting data into train and test sets####
        #### normalise
        
        X_train, X_test, y_train_minus, y_test_minus = \
        train_test_split(sc.fit_transform(X), y_minus, test_size=0.25, random_state=0) #splitting training and test labels for y_minus
        
        X_train, X_test, y_train_plus, y_test_plus = \
        train_test_split(sc.fit_transform(X), y_plus, test_size=0.25, random_state=0) #splitting training and test labels for y_minus
        
#         n_classes =y.shape[1]
#         n_samples, n_features = X.shape
#         #### normalise
#         X_train, X_test, y_train, y_test = \
#         train_test_split(sc.fit_transform(X), y, test_size=0.1, random_state=0) #splitting training and test labels for y_minus
# #         parameters = {'kernel':('linear', 'rbf'),'gamma' : np.logspace(-2, 1, 5), 'C':[1, 10]}
        clf = svm.SVC(kernel ='rbf', C=10, gamma='auto', random_state=0, class_weight="balanced", cache_size=1024)
        clf.fit(X_train, y_train_plus)
        #print 'Best parameters %s' %clf.best_params_
        #print 'Cross validation accuracy: mean = %0.3f' % clf.best_score_
        t0 = time.time()
        scores = cross_val_score(clf, X_train, y_train_plus, cv=5,scoring='accuracy', n_jobs=-1)
        #print key +1
        #print "Training Set"
        #print "SVC with rbf kernel -> cross validation accuracy: mean = %0.3f std = %0.3f" % (np.mean(scores), np.std(scores))
        #print "Test Set"
        
        y_pred = clf.predict(X_test)
        #print clf.score(X_test, y_test_plus)  
        test_mse =mean_squared_error(y_test_plus, y_pred)
        test_fbeta= fbeta_score(y_test_plus, y_pred, average='macro', beta=0.5)
        accuracy_ = accuracy_score(y_test_plus, y_pred)
        cnf_matrix=confusion_matrix(y_test_plus, y_pred).astype('float')
        cnf_matrix =cnf_matrix/cnf_matrix.sum(axis=1) #normalised confusion matrix
        #print "R2_score: %3f" %r2_score(y_test_plus, y_pred)
        results_list.append({'Features':df2[key][j].columns.values,'accuracy':accuracy_,'confusion matrix':cnf_matrix,'test_mse':test_mse, 'fbeta':test_fbeta})
        ##### parameters###

print 'Time passed:'
print time.time()- t0
print '####'

In [ ]:
len(results_list)
type(results_list)
list_df=pd.DataFrame(results_list)

***Other Classifiers for Comparison***

In [ ]:
X=sc.fit_transform(df_features_all)

In [ ]:
pca=PCA(n_components=5)
S_pca_ = pca.fit(X).transform(X)

ica = FastICA(n_components=4)
S_ica_ = ica.fit_transform(X)  # Estimate the sources

S_ica_ /= S_ica_.std(axis=0)
A_ = ica.mixing_  # Get estimated mixing matrix


###PCA
np.set_printoptions(precision=3, suppress=True)
pca.fit(X)
print(pca.explained_variance_ratio_)
var_exp= pca.explained_variance_ratio_

In [ ]:
def plot_samples(S, axis_list=None):
    plt.scatter(S[:, 0], S[:, 1], s=2, marker='o', zorder=10,
                color='steelblue', alpha=0.5)
    if axis_list is not None:
        colors = ['orange', 'red']
        for color, axis in zip(colors, axis_list):
            axis /= axis.std()
            x_axis, y_axis = axis
            # Trick to get legend to work
            plt.plot(0.1 * x_axis, 0.1 * y_axis, linewidth=2, color=color)
            plt.quiver(0, 0, x_axis, y_axis, zorder=11, width=0.01, scale=6,
                       color=color)

    plt.hlines(0, -3, 3)
    plt.vlines(0, -3, 3)
    plt.xlim(-3, 3)
    plt.ylim(-3, 3)
    plt.xlabel('x')
    plt.ylabel('y')

In [ ]:
plt.figure()
plt.subplot(2, 2, 1)
plot_samples(X / X.std())
plt.title('True Independent Sources')
axis_list = [pca.components_.T, ica.mixing_]
plt.subplot(2, 2, 2)
#plot_samples(S_ica_ / S_ica_.std(), axis_list=axis_list)
#legend = plt.legend(['PCA', 'ICA'], loc='upper right')
#legend.set_zorder(100)

In [ ]:
plt.scatter(S_ica_[:, 0]/S_ica_.std(), S_ica_[:, 1]/S_ica_.std(), s=2, marker='o', zorder=10,
                color='steelblue', alpha=0.5)

In [ ]:
class TrainerSVM:
    def __init__(self, training_data, training_labels,kernel_type):
        self.training_data_= training_data
        self.training_labels_ = training_labels
        self.kernel_type_ = kernel_type
        self.param_grid_ = {'C':[1, 10]} #once code is stable moving over to using this
        
    def hyperparam_tune(self):
        pipe_svc = Pipeline([('scl', max_abs_scaler),('clf', SVC(random_state=1))])
        param_range = [0.1, 1.0, 10.0, 100.0]
        param_grid = [{'clf__C': param_range,'clf__gamma': param_range,'clf__kernel': ['rbf']}]
        gs = GridSearchCV(estimator=pipe_svc,param_grid=param_grid,scoring='accuracy',cv=10,n_jobs=-1)
        gs= gs.fit(self.training_data_, self.training_labels_)
        tuned_param_= gs.best_params_
        return tuned_param_
    
    def tuned_predictor(self):
        C_tuned_=self.hyperparam_tune()['clf__C']
        gamma_tuned_=self.hyperparam_tune()['clf__gamma']
        kernel_best_ =self.hyperparam_tune()['clf__kernel']
        best_clf_ = svm.SVC(C=C_tuned_, cache_size=200, class_weight='balanced', coef0=0.0,
                            decision_function_shape=None, degree=3, gamma=gamma_tuned_, kernel=kernel_best_,
                            max_iter=-1, probability=False, random_state=None, shrinking=True,
                            tol=0.001, verbose=False)
        best_clf_.fit(self.training_data_, self.training_labels_)
        return best_clf_
           
        
    def learn_best_parameters(self):
            
            svc = svm.SVC(kernel=self.kernel_type_)
            grid = GridSearchCV(svc, param_grid=self.param_grid_)
            grid.fit(self.training_data_, self.training_labels_)
            svc.fit(self.training_data_, self.training_labels_)
            print("The best parameters are %s with a score of %0.2f"
                % (grid.best_params_, grid.best_score_))
            train_result_=svc.predict(self.training_data_)
            if self.kernel_type_ =='linear':
                print 'Linear Kernel Results'
                print classification_report(self.training_labels_, train_result_)
                print confusion_matrix(self.training_labels_, train_result_)
                
            
            elif self.kernel_type_ == 'rbf':
                print 'RBF Kernel Results'
                print classification_report(self.training_labels_, train_result_)
                confmat= confusion_matrix(self.training_labels_, train_result_)
                fig, ax = plt.subplots(figsize=(2.5, 2.5))
                ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
                for i in range(confmat.shape[0]):
                    for j in range(confmat.shape[1]):
                        ax.text(x=j, y=i,s=confmat[i, j],va='center', ha='center')
                plt.xlabel('predicted label')
                plt.ylabel('true label')
                plt.show()


def my_kernel(X, Y):
    """
    We create a custom kernel:

                 (2  0)
    k(X, Y) = X  (    ) Y.T
                 (0  1)
    """
    M = np.array([[2, 0], [0, 1.0]])
    return np.dot(np.dot(X, M), Y.T)

In [ ]:
start_time = time.time()
test_object=TrainerSVM(X_train, y_train_minus, 'rbf').hyperparam_tune()
print("--- %s seconds ---" % (time.time() - start_time))
%time test_object

In [ ]:
test_object.keys()
#test_object['clf__C']
#test_object

In [ ]:
test_object

In [ ]:
clf_predictor=TrainerSVM(X_train, y_train_minus,'linear').tuned_predictor()

In [ ]:
clf_labels =clf_predictor.predict(X_test)
print clf_labels
print y_test

In [ ]:
#print classification_report(y_train, clf_labels)

In [ ]:
from sklearn.externals import joblib
joblib.dump(clf_predictor, 'sample_predictor.pkl')

In [ ]:
os.listdir('/home/ak/Documents/Research/QFPaper/aknotebooks')

In [ ]:
saved_clf = joblib.load('sample_predictor.pkl')

In [ ]:
saved_clf

In [ ]:
no_i=2
df2= [df_features_all[list(pair)]for pair in list(iter.combinations(df_features_all, no_i)) ]
#take all the combination of features in pairs
#print df2

In [ ]:
key=1
train_labels =data_dic[fkeys[key]].dropna()['trend_label_quantiles']
len(train_labels)

In [ ]:
len(df2[90])

In [ ]:
k_len= 10 #len(df2)
for j in range(0,k_len):
    df2= [df_features_all[list(pair)]for pair in list(iter.combinations(df_features_all, no_i)) ] 
    #take all the combination of features in pairs
    print df2

In [ ]:
train_labels =data_dic[fkeys[key]].dropna()['trend_label_quantiles']
    diff= len(df_features_all) -len(train_labels)
    
    #########
    X= df_features_all.iloc[diff:]
    n_features = X.shape[1]
    
    y= train_labels
    n_classes = train_labels.shape[0]
    y_minus = (y==-1).astype(int)
    y_zeros = (y==0).astype(int)
    y_plus = (y==1).astype(int)
    ####splitting data into train and test sets####
    #### normalise
    X_train, X_test, y_train_minus, y_test_minus = \
    train_test_split(sc.fit_transform(X), y_minus, test_size=0.25, random_state=0) #splitting training and test labels for y_minus
    X_train, X_test, y_train_plus, y_test_plus = \
    train_test_split(sc.fit_transform(X), y_plus, test_size=0.25, random_state=0) #splitting training and test labels for y_minus
    t0 = time.time()
    print t0